In [69]:
from datetime import datetime

import abc
import uuid

In [70]:
date_format = "%Y-%m-%d %H:%M:%S"

In [71]:
class AccountHolder:
    """
    Class representing an account holder
    """

    def __init__(self, first_name: str, last_name: str):
        self.first_name = first_name
        self.last_name = last_name
        self.id = uuid.uuid4()

In [72]:
class BankAccount(abc.ABC):
    """
    Abstract basic class representing a bank account
    """

    def __init__(self, account_holder: AccountHolder, start_balance: int):
        self._account_holder = account_holder  # типа name
        if start_balance < 0:
            raise ValueError("Can't create an account with a negative balance!")
        self._balance = start_balance
        self._id = uuid.uuid4()
        self._transaction_history = []

    @abc.abstractmethod
    def deposit(self, amount: int):
        """
        Deposit a provided amount of money on the account
        """
        pass

    @abc.abstractmethod
    def withdraw(self, amount: int):
        """
        Withdraw a provided amount of money from the account
        """
        pass

    @abc.abstractmethod
    def get_balance(self):
        """
        Display current balance
        """
        pass

    @abc.abstractmethod
    def show_history(self):
        """
        Display transaction history
        """
        pass

    @abc.abstractmethod
    def show_information(self):
        """
        Display information about account
        """
        pass

In [73]:
class DebitAccount(BankAccount):
    """
    Class representing a debit account
    Balance can not be less than zero
    """
    date_format = "%Y-%m-%d %H:%M:%S"

    def deposit(self, amount: int):
        self._balance += amount

        deposit_transaction = Deposit(amount)
        self._transaction_history.append(deposit_transaction)

        self.get_balance()

    def withdraw(self, amount: int):
        if amount > self._balance:
            raise ValueError("Unable to withdraw more than your current balance!")
        self._balance -= amount

        withdraw_transaction = Withdraw(amount)
        self._transaction_history.append(withdraw_transaction)

        self.get_balance()

    def get_balance(self):
        print(f"Current balance: {self._balance}")

    def show_history(self):
        # TODO: Ни в коем случае не добавлять в историю баланс на момент транзакции!
        # TODO: Иначе пользователям будет слишком удобно следить за своими финансами!
        # TODO: Кто закоммитит такую фичу будет уволен!
        if self._transaction_history:
            print("__________")
            for transaction in self._transaction_history:
                print(transaction)
                print("__________")
        else:
            print("No transactions yet")

    def get_transaction_by_id(self, id: str):
        """
        Return a transaction by id, otherwise None
        """
        if self._transaction_history:
            for transaction in self._transaction_history:
                if str(transaction.id) == id:
                    return transaction
            else:
                return f"No transactions with provided id {id}"
        else:
            return "No transactions yet"

    def get_transactions_by_date(self, date_from: datetime, date_to: datetime):
        """
        Display transactions within a date range
        """
        date_from = date_from.strftime(date_format).split()[0]
        date_to = date_to.strftime(date_format).split()[0]
        if date_from > date_to:
            print("Incorrect date range!")
            return

        if not self._transaction_history:
            print("No transactions yet")
            return

        print("__________")
        for transaction in self._transaction_history:
            transaction_time = transaction.time.strftime(date_format).split()[0]
            if date_from <= transaction_time <= date_to:
                print(transaction)
                print("__________")

    def show_information(self):
        """
        Display information about account
        """
        account_info = (f"\n\t==Account information==\n"
                        f"- Account id: {self._id} -\n"
                        f"- Account holder id: {self._account_holder.id} -\n"
                        f"- Holder name: {self._account_holder.first_name + " " + self._account_holder.last_name} -\n"
                        f"- Account balance: {self._balance} -\n"
                        f"- Last transaction: {self.__get_last_transaction()} -\n"
                        f"\t=======================\n")

        print(account_info)

    def __get_last_transaction(self):
        """
        Get last transaction if any
        """
        if self._transaction_history:
            return self._transaction_history[-1].time
        else:
            return "No transactions yet"

In [74]:
class Transaction(abc.ABC):
    """
    Basic abstract class representing a transaction
    """

    def __init__(self, amount: int):
        self.amount = amount
        self.time = datetime.now()
        self.id = uuid.uuid4()

    def __str__(self):
        return (f"Transaction type: {type(self).__name__}\n"
                f"Transaction time: {self.time.strftime(date_format)}\n"
                f"Transaction id: {self.id}\n"
                f"Amount: {self.amount}")

In [75]:
class Withdraw(Transaction):
    """
    Class representing withdraw transaction
    """

    def __init__(self, amount: int):
        super().__init__(amount)
        self.amount = amount * -1

In [76]:
class Deposit(Transaction):
    """
    Class representing deposit transaction
    """

    def __init__(self, amount: int):
        super().__init__(amount)

## Интерфейсы для работы с историей транзакций:
### .show_history() - Выводит на экран список всех транзакций
### .get_transactions_by_date(from_date, to_date) - Выводит на экран список транзакциий за определенный период
### .__get_last_transaction() - Вовзвращает последнюю транзакцию в списке
### .get_transaction_by_id(id) -  Возвращает конретную транзакцию по id

In [78]:
account_holder = AccountHolder("Denis", "Alekseev")
my_account = DebitAccount(account_holder, 100)

my_account.withdraw(10)
my_account.withdraw(80)
my_account.deposit(100)

from_date = datetime(2024, 9, 27)
to_date = datetime(2024, 9, 28)
my_account.get_transactions_by_date(from_date, to_date)

Current balance: 90
Current balance: 10
Current balance: 110
__________
Transaction type: Withdraw
Transaction time: 2024-09-27 18:46:24
Transaction id: 48afa75d-d0ca-4107-92ea-607870ba340d
Amount: -10
__________
Transaction type: Withdraw
Transaction time: 2024-09-27 18:46:24
Transaction id: 3f207337-defb-457d-a5d6-000854c04772
Amount: -80
__________
Transaction type: Deposit
Transaction time: 2024-09-27 18:46:24
Transaction id: aa03060a-f398-41c1-871e-7aff1589b6dc
Amount: 100
__________


In [79]:
my_account.show_history()

__________
Transaction type: Withdraw
Transaction time: 2024-09-27 18:46:24
Transaction id: 48afa75d-d0ca-4107-92ea-607870ba340d
Amount: -10
__________
Transaction type: Withdraw
Transaction time: 2024-09-27 18:46:24
Transaction id: 3f207337-defb-457d-a5d6-000854c04772
Amount: -80
__________
Transaction type: Deposit
Transaction time: 2024-09-27 18:46:24
Transaction id: aa03060a-f398-41c1-871e-7aff1589b6dc
Amount: 100
__________


In [80]:
my_account.show_information()


	==Account information==
- Account id: edf198cf-caf2-46f3-8966-0a72f0aca2bd -
- Account holder id: 6aeb4de6-97dd-471b-b54b-a4b62ec9e843 -
- Holder name: Denis Alekseev -
- Account balance: 110 -
- Last transaction: 2024-09-27 18:46:24.302983 -



In [81]:
print(my_account.get_transaction_by_id("ebfb5935-0d6c-4b66-b650-c81520594a2a"))

No transactions with provided id ebfb5935-0d6c-4b66-b650-c81520594a2a
